In [10]:
import os
import sys
import h5py
import pdb
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.io import loadmat

sys.path.insert(0, '../')
from ecg_AAAI.parse_dataset.readECG import loadECG
from ecg_AAAI.models.ecg_utils import get_all_adjacent_beats
from ecg_AAAI.models.supervised.ecg_fi_model_keras import build_fi_model 
from ecg_AAAI.models.supervised.ecg_fc import build_fc_model
from ecg_AAAI.models.gpu_utils import restrict_GPU_keras
from ecg_AAAI.models.supervised.eval import evaluate_AUC, evaluate_HR, risk_scores

mode = 'two_beat'
m_type = 'nn'

In [5]:
# Load Y
hf = h5py.File('datasets/data.h5', 'r')
y_train = np.array(hf.get('y_train'))
y_test = np.array(hf.get('y_test')) 
hf.close()

# Load X 
if mode == "one_beat":     
    x_file = h5py.File('datasets/one_beat.h5', 'r')
    all_test = h5py.File('datasets/all_test_one.h5', 'r')
    n_beats = 1000
    instance_length = 128
elif mode == "three_beat":
    x_file = h5py.File('datasets/three_beat.h5', 'r')
    all_test = h5py.File('datasets/all_test_three.h5', 'r')
    n_beats = 998
    instance_length = 384
elif mode == 'four_beat':
    x_file = h5py.File('datasets/four_beat.h5', 'r')
    all_test = h5py.File('datasets/all_test_four.h5', 'r')
    n_beats = 997
    instance_length = 512
else: 
    x_file = h5py.File('datasets/two_beat.h5', 'r')
    all_test = h5py.File('datasets/all_test_two.h5', 'r')
    n_beats = 999 
    instance_length = 256

In [6]:
# TODO: redo test train split
X_train = np.array(x_file.get('X_train'))
X_test = np.array(x_file.get('X_test')) 
x_file.close()

all_test_patients = np.array(all_test.get('test_patients'))
all_test_patient_labels = np.array(all_test.get('test_patient_labels'))
all_test_pids = np.loadtxt('all_test_pids')

In [7]:
len(all_test_patients)

4816

In [8]:
# Create balanced 80/20 split by moving some elements from test to train
# TODO: make this more random? /even?
train_add = all_test_patients[:4000].reshape(4000*n_beats, 1, instance_length) 
train_label_add = np.array([[g]*n_beats for g in all_test_patient_labels[:4000]])
X_train = np.concatenate([X_train,train_add])
y_train = np.concatenate([y_train, train_label_add.reshape(4000*n_beats)])

all_test_patients = all_test_patients[4000:]
all_test_patient_labels = all_test_patient_labels[4000:]
all_test_pids = all_test_pids[4000:]

# Load test patient metadata
test_hf = h5py.File('datasets/test_pids.h5', 'r')
test_pids = np.array(test_hf.get('pids'))
test_patient_labels = np.array(test_hf.get('patient_labels'))
test_hf.close()

In [9]:
# Pre-process data
input_dim = X_train.shape[2]
X_train = np.swapaxes(X_train, 1, 2)
X_test = np.swapaxes(X_test, 1, 2)

# Shuffle X_train
new_order = np.arange(len(X_train))
np.random.shuffle(new_order)
X_train = X_train[new_order]
y_train = y_train[new_order]

# Shuffle X_test
new_order = np.arange(len(X_test))
np.random.shuffle(new_order)
X_val = X_test[new_order][:3000]
y_val = y_test[new_order][:3000]

patient_outcomes = loadmat("./datasets/patient_outcomes.mat")['outcomes'] 
survival_dict = {x[0]: x[4] for x in patient_outcomes}
print("Done loading data...")

Done loading data...


# Dataset Summary

    ### Original dataset: n = 6354 \\  [6067, 287]
    ### Paul dataset: n = 4786 \\ [5104, 132]

    ### txt files: 4985
    ### then in total the number of patients is 4975... weird

627

In [19]:
n_train = float(len(y_train))
n_test = float(len(y_test))
n = float(n_train + n_test)
print("Train/Test %: ", n_train/n, n_test/n)
print("Train/Test n:", n_train, n_test)

Train/Test %:  0.8689928959465106 0.13100710405348934
Train/Test n: 4159000.0 627000.0


In [20]:
n_pos_train = len(np.where(y_train == 1)[0])
pct_pos_train = n_pos_train/float(len(y_train))

n_pos_test = len(np.where(y_test == 1)[0])
pct_pos_test = n_pos_train/float(len(y_test))

                
print("Train \t p(y = 1): ", pct_pos_train)
print("Test \t p(y = 1):", pct_pos_test)

Train 	 p(y = 1):  0.04832892522240923
Test 	 p(y = 1): 0.32057416267942584


# Model Building

In [22]:
if m_type == 'LR':
    n_iter = 1
    m = LogisticRegression()

    X_train = np.squeeze(X_train, 2)
    test_patients = np.resize(X_test, (627, 1000, input_dim))
    train_m = lambda : m.fit(X_train, y_train)
    score_m = lambda test_patients: risk_scores(m, test_patients)

else:
    n_iter = 5
    m, embedding_m = build_fc_model((input_dim, 1))
    m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    m.summary()

    test_patients = np.resize(X_test, (627, 1000, input_dim, 1))
    class_weight = {0: 1.,
                    1: 1/pct_pos_train}
    train_m = lambda : m.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=10, verbose=True, batch_size=10000, class_weight=class_weight)
    score_m = lambda test_patients: risk_scores(m, test_patients)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 128, 1)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)               0         
_________________________________________________________________
fc_1 (Dense)                 (None, 2)                 258       
_________________________________________________________________
softmax (Dense)              (None, 1)                 3         
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


# Model Training

In [23]:
hrs = []
discrete_hrs = []
auc_vals = []
for i in range(n_iter):
    train_m()
    
    all_scores = score_m(all_test_patients)
    cutoff = np.percentile(all_scores, 75)
    all_discrete_scores = [1 if x >= cutoff else 0 for x in all_scores]

    all_auc = evaluate_AUC(all_scores, all_test_patient_labels)
    all_hr = evaluate_HR(survival_dict, scores, all_test_pids, all_test_patient_labels, "continous")
    all_discrete_hr = evaluate_HR(survival_dict, all_discrete_scores, all_test_pids, all_test_patient_labels, "discrete")
    
    hrs.append(all_hr)
    discrete_hrs.append(all_discrete_hr)
    auc_vals.append(all_auc)
    
    print("ALL patient scores: ")
    print(all_auc,all_hr, all_discrete_hr)

Train on 4159000 samples, validate on 3000 samples
Epoch 1/10
4159000/4159000 [==============================] - 14s 3us/step - loss: 1.2917 - acc: 0.5061 - val_loss: 0.7032 - val_acc: 0.6900
Epoch 2/10
4159000/4159000 [==============================] - 12s 3us/step - loss: 1.2512 - acc: 0.6717 - val_loss: 0.6930 - val_acc: 0.6867
Epoch 3/10
4159000/4159000 [==============================] - 13s 3us/step - loss: 1.2373 - acc: 0.6855 - val_loss: 0.6799 - val_acc: 0.7133
Epoch 4/10
4159000/4159000 [==============================] - 13s 3us/step - loss: 1.2307 - acc: 0.6923 - val_loss: 0.6779 - val_acc: 0.7087
Epoch 5/10
4159000/4159000 [==============================] - 13s 3us/step - loss: 1.2272 - acc: 0.6945 - val_loss: 0.6719 - val_acc: 0.7177
Epoch 6/10
4159000/4159000 [==============================] - 14s 3us/step - loss: 1.2250 - acc: 0.6955 - val_loss: 0.6758 - val_acc: 0.7080
Epoch 7/10
4159000/4159000 [==============================] - 14s 3us/step - loss: 1.2235 - acc: 0.6953

# Model Evaluation

In [24]:
all_scores = score_m(all_test_patients)
cutoff = np.percentile(all_scores, 75)
all_discrete_scores = [1 if x >= cutoff else 0 for x in all_scores]

all_auc = evaluate_AUC(all_scores, all_test_patient_labels)
all_hr = evaluate_HR(survival_dict, scores, all_test_pids, all_test_patient_labels, "continous")
all_discrete_hr = evaluate_HR(survival_dict, all_discrete_scores, all_test_pids, all_test_patient_labels, "discrete")

In [32]:
len(all_test_patients)

816

# Model Visualization

In [ ]:
# Use embedding_m to visualize
    # which patients are predicted really correctly
    # which patients are predicted very wrong
    # what kinds of instances are very strong indicators
    # what kinds of instances are a strong anti-indicators


# Dataset Verification

In [16]:
split_num = 0
for split_num in range(5):
    split_train_fname = "./datasets/splits/split_" + str(int(split_num)) + "/train.h5"
    split_test_fname = "./datasets/splits/split_" + str(int(split_num)) + "/test.h5"

    try:
        #print(h5py.File(split_train_fname, "r").get("adjacent_beats").shape)
        print(h5py.File(split_train_fname, "r").get("pids").shape)

    except:
        print("Didn't work: ", split_num)

(4046,)
(4046,)
(4046,)
(4046,)
(4046,)


In [ ]:
y_mode = "cvd"
splits = ["0", "1", "2", "3", "4"]
split_num = "0"
split_dir = "./datasets/splits/split_" + split_num

# Load Y
y_train = np.loadtxt(split_dir + "/" + y_mode + "_train_labels")
y_test = np.loadtxt(split_dir + "/" + y_mode + "_test_labels")


x_train_file = h5py.File(split_dir + "/train.h5")
x_test_file = h5py.File(split_dir + "/test.h5")
X_train = np.array(x_train_file.get('adjacent_beats'))
X_test = np.array(x_test_file.get('adjacent_beats'))
x_train_file.close()
x_test_file.close()

--Call--
> /home/divyas/.local/lib/python3.6/site-packages/IPython/core/displayhook.py(247)__call__()
-> def __call__(self, result=None):
